<a href="https://colab.research.google.com/github/dedemasutti/AI-projects/blob/master/detec%C3%A7%C3%A3o_de_distanciamento_social.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###CONFIGURANDO OS VALORES DAS VARIAVÉIS

In [1]:
MIN_CONFIDENCE = 0.5
MIN_THRESHOLD = 0.5

In [2]:
# DEFINE A DISTANCIA MÍNIMA SEGURA EM PIXELS(two or more peoples)
MIN_DISTANCE = 50

### CROWD DETECTOR 

In [3]:
import numpy as np
import cv2

In [ ]:
def crowd_detector(frame, net, ln, personIdx = 0):
  #pega as dimensões do frame e inicia lista de resultados
  (H,W) = frame.shape[:2]
  results = []

  blob = cv2.dnn.blobFromImage(frame, 1/255.0(416,416, swapRB = True, crop =False))
  net.setInput(blob)
  layerOutputs = net.forward(ln)

  boxes = []
  centroids = []
  confidences = []

# loop over each of the layers output
  for output in layerOutputs:

# loop over each of the detections
    for detection in output:
      scores = detection[5:]
      classID = np.argmax(scores)
      confidence = scores[classID]

      if classID == personIdx and confidence > MIN_CONFIDENCE:
        box = detection[0:4 * np.array[W,H,W,H]
        (centerX, centerY, width, height = box.astype('int')


        x = int(centerX - (width / 2))
        y = int(centerY -(height / 2))
      
        boxes.append([x, y, int(width), int(height)])
        centroids.append((centerX, centerY))
        confidences.append(float(confidence))


idxs = cv2.dnn.NMSBoxes(boxes,confidences,MIN_CONFIDENCE,MIN_THRESHOLD)

if len(idxs) > 0:
  for i in idxs.flatten():
    (x, y) = (boxes[i][0], boxes[i][1])
    (w, h) = (boxes[i][2], boxes[i][3])

    r = (confidences[i], (x, y, x + w, y + h), centroids[i]
    results.append(r)

    return results


###GRAB FRAMES FROM VIDEO AND MAKE PREDICTION MEASURING DISTANCES OF DETECTED PEOPLE 

In [ ]:
#imports 
from google.colab.patches import cv2_imshow
from spicy.spatial import distance as dist
import numpy as np
import argparse
import os
import cv2
import imutils


In [ ]:
#Construct the argument parse and parse the arguments
from numpy.core.fromnumeric import shape
from traitlets.traitlets import default
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--input",type = str, default="",
    help = "path to (optional)input video file")
ap.add_argument("-0", "--output", type = str, default = "",
    help = "path to(optional) output video file")
ap.add_argument("-d", "--display", type = str, default = "",
    help = "whether or not output frame should be displayed")
args = vars(ap.parse_args(["--input","file path", "--output","my_output.avi","--display","1"]))

# Load the COCO class labels
labelsPath = os.path.sep.join(["file path"])
LABELS = open(labelsPath).read().strip().split("\n\")

#derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join(["file yolo.weights path"])
configPath = os.path.sep.join(["file yolo.config path"])

# Load our YOLO object detector trained on Dataset
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(weightsPath, configPath)

# Determine only the output layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] -1] for i in net.getUnconnectedOutLayers()]

#Initialize the video stream and pointer to output video file
print("[INFO] accessing video stream...")
vs = cv2.VideoCapture(args["input"] if args["input"] else 0)
writer = None

# loop over the frames from the video stream
while True:
# read the next frame from the file
  (grabbed, frame) = vs.read()
# if the frame was not grabbed ,then we have reached the end of the stream
  if not grabbed:
    break

# resize the frame and then detect people
frame = imutils.resize(frame, width = 500)
results = detect_people(frame, net, ln, personIdx=LABELS.index("person"))

# Initialize the set of indexes that violate minimum social distance
violate = set()

# ensure there are at least two people 
#detected(required in order to compute our pairwise distance maps)
if len(results) >= 2:
  #extract all centroids from the results and 
  #compute the euclidean distance between all pairs of the centroids
  centroids = np.array([r[2] for r in results])
  d = dist.cdist(centroids, centroids, metric = "euclidean")
  #loop over the upper triangular distance matrix
  for i in range(0, d.shape[0]):
    for j in range(i + 1, d.shape[1]):
      #check if the distance between any two centroid
      #pairs are less than the configured number of pixels
      if d[i,j] < MIN_DISTANCE:
        #update our violation set with the indexes of centroid pairs
        violate.add(i)
        violate.add(j)

# loop over the results
for (i(prob,bbox, centroid)) in enumerate(results):
  # extract bounding box and centroid coordinates then initialize 
  # the color of annotation
  (startX, startY, endX, endY) = bbox
  (cX, cY) = centroid
  color = (0,255,0)
  # if the index pair exists whithin the violation set
  # update the color
  if i in violate:
    color = (0,0,255)

#draw(1) a bounding box around person and (2)
#the centroid coordinates of the person.
cv2.rectangle(frame(startX, startY),(endX, endY),color, 2)
cv2.circle(frame)

#draw the total number of social distance violation
#on the output frame
text = "alerta para distanciamento social": {}.format(len(violate))
cv2.putText(frame, text,(10,frame.shape[0] - 25),
            cv2.FONT_HERSHEY_DUPLEX, 0.75,(0,0,255),3)

# check to see if the output frame should be displayed to our screen
if args["display"] > 0:
  #show the output frame
  cv2.imshow(frame)
  key = cv2.waitKey(1) & 0xFF

  # 'q' key should pressed to break the loop
  if key == ord("q"):
    break
  #if an output video file path has been supplied and the
  # videos writer has not been initialized. 
  if args["output"] != "" and writer is None:
    #initialize our video writer
    fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    writer = cv2.VideoWriter(args["output"], fourcc, 25, frame.shape[1],
                             frame.shape[0], True)
    #if the video writer is not None write the frame to the output video file
    if writer is not None:
      writer.write(frame)